### Setup

In [64]:
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import numpy as np
import numpy
import math
import timeit
import matplotlib.pyplot as plt
from tensorflow.keras import backend as k
from tensorflow.keras import layers
from tensorflow.keras import Model

In [65]:
USE_GPU = False

if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'



In [66]:
"""temp_config"""


batch_size = 1


k = 16
k2 = 8
output_size = 7
output_size_valuehead = 10
#batch_size = 64
input_dims = [batch_size, 6, 7, 8]
new_dims = [batch_size, 6 * 7 * 8]
input_prodsize = 8 * 6 * 7
after_input_dims = [batch_size, 6, 7, 8]
after_new_dims = [batch_size, 6 * 7 * 8]

### Model definition

In [67]:
def two_layer_fc_functional(inputs, hidden_size, num_classes):     
    initializer = tf.variance_scaling_initializer(scale=2.0)
    flattened_inputs = tf.layers.flatten(inputs)
    fc1_output = tf.layers.dense(flattened_inputs, hidden_size, activation=tf.nn.relu,
                                 kernel_initializer=initializer)
    scores = tf.layers.dense(fc1_output, num_classes,
                             kernel_initializer=initializer)
    return scores



        


In [68]:
def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    tf.reset_default_graph()
    input_size, hidden_size, num_classes = 50, 42, 10

    # As usual in TensorFlow, we first need to define our computational graph.
    # To this end we first construct a two layer network graph by calling the
    # two_layer_network() function. This function constructs the computation
    # graph and outputs the score tensor.
    with tf.device(device):
        x = tf.zeros((64, input_size))
        scores = two_layer_fc_functional(x, hidden_size, num_classes)

    # Now that our computational graph has been defined we can run the graph
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        scores_np = sess.run(scores)
        print(scores_np.shape)

In [69]:
#test_two_layer_fc_functional()

### New Model


In [70]:
class ConvNet(tf.keras.Model):
    def __init__(self, channel_1):
        super().__init__()
        initializer = tf.variance_scaling_initializer(scale=2.0) 
          
        self.conv1 = tf.layers.Conv2D(channel_1, (3,3), (1,1), padding='same',
                                      kernel_initializer=initializer,
                                      activation=None)
        
    def call(self, x, training=None):
        scores = None
        
        scores = self.conv1(x)
            
        return scores

In [71]:
def DENSE_block(inputs, num_units, kernel_initializer, activation, input_dims, new_dims):
    l_re_1 = tf.reshape(inputs, new_dims)
    #might want to add initializer
    l_re_2 = tf.layers.dense(l_re_1, num_units, activation)
    return l_re_2

In [72]:
def ADD_bias_block(inputs, train_bool, input_dims):
    
    
    
    l_hid_re = tf.reshape(inputs, new_dims)
    l_hid_1 = tf.layers.dense(inputs, 1, activation = None)
    l_hid_2 = tf.math.scalar_mul(0, l_hid_1)
    l_hid_3 = tf.layers.dense(l_hid_re, input_prodsize, activation = None)
    l_hid_4 = tf.reshape(l_hid_3, input_dims)
    return l_hid_4

In [73]:
def BASE_block(inputs, train_bool, initializer):
    #l_hidden_1 = tf.layers.conv2d(inputs, k, (3,3), (1,1), 'same', kernel_initializer=initializer, activation = None)
    l_hidden_1 = tf.layers.conv2d(inputs, k, (3,3), (1,1), 'same', kernel_initializer=initializer, activation = None)
    #l_hidden_2 = tf.layers.batch_normalization(l_hidden_1, training = train_bool)
    l_hidden_3 = tf.nn.relu(l_hidden_1)
    return l_hidden_3

        
    


In [74]:
def RESIDUAL_block(inputs, train_bool, initializer):
    l_resid_1 = tf.layers.conv2d(inputs, k, (3,3), (1,1), 'same', kernel_initializer=initializer, activation = None)
    #l_resid_2 = tf.layers.batch_normalization(l_resid_1, training = train_bool)
    l_resid_3 = tf.nn.relu(l_resid_1)
    l_resid_4 = tf.layers.conv2d(l_resid_3, k, (3,3), (1,1), 'same', kernel_initializer=initializer, activation = None)
    #l_resid_5 = tf.layers.batch_normalization(l_resid_4, training = train_bool)
    l_resid_6 = tf.keras.layers.Add()([l_resid_1, l_resid_4])
    l_resid_7 = tf.nn.relu(l_resid_6)
    return l_resid_7


In [75]:
def RESIDUAL_tower(inputs, train_bool, initializer):
    l_resid_1 = RESIDUAL_block(inputs, train_bool, initializer)
    l_resid_2 = RESIDUAL_block(l_resid_1, train_bool, initializer)
    l_resid_3 = RESIDUAL_block(l_resid_2, train_bool, initializer)
    l_resid_4 = RESIDUAL_block(l_resid_3, train_bool, initializer)
    l_resid_5 = RESIDUAL_block(l_resid_4, train_bool, initializer)
    l_resid_6 = RESIDUAL_block(l_resid_5, train_bool, initializer)
    return l_resid_6


In [76]:
"""def ACTION_head(inputs, train_bool, initializer):
    l_action_1 = tf.layers.conv2d(inputs, k2, (1,1), (1,1), 'same', kernel_initializer=initializer, activation = None)
    l_action_2 = ADD_bias_block(l_action_1, train_bool, input_dims)
    l_action_3 = tf.layers.batch_normalization(l_action_2, training = train_bool)
    l_action_4 = tf.nn.relu(l_action_3)
    l_action_5 = DENSE_block(l_action_4, 7, initializer, tf.nn.softmax, after_input_dims, after_new_dims)
    return l_action_5"""
    
def ACTION_head(inputs, train_bool, initializer):
    l_action_1 = tf.layers.conv2d(inputs, k2, (1,1), (1,1), 'same', kernel_initializer=initializer, activation = None)
    #l_action_3 = tf.layers.batch_normalization(l_action_1, training = train_bool)
    l_action_4 = tf.nn.relu(l_action_1)
    l_action_5 = DENSE_block(l_action_4, 7, initializer, tf.nn.softmax, after_input_dims, after_new_dims)
    return l_action_5

In [77]:
"""def VALUE_head(inputs, train_bool, initializer):
    l_value_1 = tf.layers.conv2d(inputs, k2, (1,1), (1,1), 'same', kernel_initializer=initializer, activation = None)
    l_value_2 = ADD_bias_block(l_value_1, train_bool, input_dims)
    l_value_3 = tf.layers.batch_normalization(l_value_2, training = train_bool)
    l_value_4 = tf.nn.relu(l_value_3)
    l_value_5 = DENSE_block(l_action_4, 10, initializer, tf.nn.softmax, after_input_dims, after_new_dims)
    l_value_6 = tf.layers.dense(l_value_5, 1, nonlinearity = tf.nn.sigmoid)
    return l_value_6"""
    
def VALUE_head(inputs, train_bool, initializer):
    l_value_1 = tf.layers.conv2d(inputs, k2, (1,1), (1,1), 'same', kernel_initializer=initializer, activation = None)
    #l_value_3 = tf.layers.batch_normalization(l_value_1, training = train_bool)
    l_value_4 = tf.nn.relu(l_value_1)
    l_value_5 = DENSE_block(l_action_4, 10, initializer, tf.nn.softmax, after_input_dims, after_new_dims)
    l_value_6 = tf.layers.dense(l_value_5, 1, nonlinearity = tf.nn.sigmoid)
    return l_value_6

In [78]:
def RESIDUAL_net_action_1(inputs, train_bool):
    #might change this
    initializer = tf.variance_scaling_initializer(scale=2.0)
    l_block_1 = BASE_block(inputs, train_bool, initializer)
    l_block_2 = RESIDUAL_tower(l_block_1, train_bool, initializer)
    l_block_3 = ACTION_head(l_block_2, train_bool, initializer)
    return l_block_3


def RESIDUAL_net_action_2(inputs, train_bool):
    #might change this
    initializer = tf.variance_scaling_initializer(scale=2.0)
    l_block_1 = BASE_block(inputs, train_bool, initializer)
    l_block_2 = RESIDUAL_tower(l_block_1, train_bool, initializer)
    l_block_3 = ACTION_head(l_block_2, train_bool, initializer)
    return l_block_3
    

In [79]:
def RESIDUAL_net_value_1(inputs, train_bool):
    #might change this
    initializer = tf.variance_scaling_initializer(scale=2.0)
    l_block_1 = BASE_block(inputs, train_bool, initializer)
    l_block_2 = RESIDUAL_tower(l_block_1, train_bool, initializer)
    l_block_3 = VALUE_head(l_block_2, train_bool, initializer)
    return l_block_3


def RESIDUAL_net_value_2(inputs, train_bool):
    #might change this
    initializer = tf.variance_scaling_initializer(scale=2.0)
    l_block_1 = BASE_block(inputs, train_bool, initializer)
    l_block_2 = RESIDUAL_tower(l_block_1, train_bool, initializer)
    l_block_3 = VALUE_head(l_block_2, train_bool, initializer)
    return l_block_3
    

In [80]:
#label = [1] * batch_size
def test_RESIDUAL_net_action_1(label):
    
    tf.reset_default_graph()
    
    model = RESIDUAL_net_action_1
    with tf.device(device):
        x = tf.zeros((1, 6, 7, 2), dtype=tf.dtypes.float32)
        #x = np.array([[[[0]*7]*6]*2]*1)
        scores = model(x, True)

    # Now that our computational graph has been defined we can run the graph
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        scores_np = sess.run(scores)
        
        
        
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = label, logits=scores)
        
        loss = tf.reduce_mean(loss)
        
        
        
        varis = tf.trainable_variables()
        params = varis
        
        
        grad_params = tf.gradients(loss, params)
        
        
        #print(scores_np.shape)
        #print(grad_params)
        return (params, grad_params)
        

In [81]:
test_RESIDUAL_net_action_1([1])

([<tf.Variable 'conv2d/kernel:0' shape=(3, 3, 2, 16) dtype=float32_ref>,
  <tf.Variable 'conv2d/bias:0' shape=(16,) dtype=float32_ref>,
  <tf.Variable 'conv2d_1/kernel:0' shape=(3, 3, 16, 16) dtype=float32_ref>,
  <tf.Variable 'conv2d_1/bias:0' shape=(16,) dtype=float32_ref>,
  <tf.Variable 'conv2d_2/kernel:0' shape=(3, 3, 16, 16) dtype=float32_ref>,
  <tf.Variable 'conv2d_2/bias:0' shape=(16,) dtype=float32_ref>,
  <tf.Variable 'conv2d_3/kernel:0' shape=(3, 3, 16, 16) dtype=float32_ref>,
  <tf.Variable 'conv2d_3/bias:0' shape=(16,) dtype=float32_ref>,
  <tf.Variable 'conv2d_4/kernel:0' shape=(3, 3, 16, 16) dtype=float32_ref>,
  <tf.Variable 'conv2d_4/bias:0' shape=(16,) dtype=float32_ref>,
  <tf.Variable 'conv2d_5/kernel:0' shape=(3, 3, 16, 16) dtype=float32_ref>,
  <tf.Variable 'conv2d_5/bias:0' shape=(16,) dtype=float32_ref>,
  <tf.Variable 'conv2d_6/kernel:0' shape=(3, 3, 16, 16) dtype=float32_ref>,
  <tf.Variable 'conv2d_6/bias:0' shape=(16,) dtype=float32_ref>,
  <tf.Variable 'co

### Grad Loop

In [ ]:
#label = [1] * 1

def loop_model(x_np, y_np, train_bool, model, params1_v):
    
    tf.reset_default_graph()
    pipe = test_RESIDUAL_net_action_1([1])
    loop_model_params_set(train_bool, model, params1_v)
    
    
  
    #model = RESIDUAL_net_action_1
    
    with tf.device(device):
        
        #change shapes of placeholders
        x = tf.placeholder(tf.float32, [None, 6, 7, 2])
        y = tf.placeholder(tf.int32, [None])
        is_training = tf.placeholder(tf.bool, name='is_training')
        
        
        scores = model(x, is_training)
        
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=scores)
        loss = tf.reduce_mean(loss)
        
        varis = tf.trainable_variables()
        params = varis
        
        
        grad_params = tf.gradients(loss, params)
        
        
        
    
    # Now that our computational graph has been defined we can run the graph
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        feed_dict = {x: x_np, y: y_np, is_training: train_bool}
        loss_np, params_np, grad_params_np = sess.run([loss, params, grad_params], feed_dict=feed_dict)

        return (loss_np, params_np, grad_params_np)
    

    

In [ ]:

def loop_model_loss(x_np, y_np, train_bool, model, params1_v):
    
    
    
    tf.reset_default_graph()
    pipe = test_RESIDUAL_net_action_1([1])
    loop_model_params_set(train_bool, model, params1_v)
    
    
    
   
    
    
    #model = RESIDUAL_net_action_1
    
    with tf.device(device):
        
        #change shapes of placeholders
        x = tf.placeholder(tf.float32, [None, 6, 7, 2])
        y = tf.placeholder(tf.int32, [None])
        is_training = tf.placeholder(tf.bool, name='is_training')
        
        
        scores = model(x, is_training)
        
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=scores)
        loss = tf.reduce_mean(loss)
        
        
        varis = tf.trainable_variables()
        params = varis
        
        
        grad_params = tf.gradients(loss, params)
        
        
    
    # Now that our computational graph has been defined we can run the graph
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        feed_dict = {x: x_np, y: y_np, is_training: train_bool}
        loss_np, params_np = sess.run([loss, params], feed_dict=feed_dict)

        return (loss_np)
    


In [ ]:

def loop_model_grad(x_np, y_np, train_bool, model, params1_v):
    
    
    tf.reset_default_graph()
    
    
    loop_model_params_set(train_bool, model, params1_v)
    
    
    
    
    #model = RESIDUAL_net_action_1
    
    with tf.device(device):
        
        #change shapes of placeholders
        x = tf.placeholder(tf.float32, [None, 6, 7, 2])
        y = tf.placeholder(tf.int32, [None])
        is_training = tf.placeholder(tf.bool, name='is_training')
        
        
        scores = model(x, is_training)
        
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=scores)
        loss = tf.reduce_mean(loss)
        
        varis = tf.trainable_variables()
        params = varis
        
        
        grad_params = tf.gradients(loss, params)
        #print(grad_params)
        
        
    
    # Now that our computational graph has been defined we can run the graph
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        feed_dict = {x: x_np, y: y_np, is_training: train_bool}
        loss_np, params_np, grad_params_np = sess.run([loss, params, grad_params], feed_dict=feed_dict)

        return grad_params_np
    


In [ ]:

def loop_model_scores(x_np, train_bool, model, params1_v):
    
    tf.reset_default_graph()
    pipe = test_RESIDUAL_net_action_1([1])
    loop_model_params_set(train_bool, model, params1_v)
    
    
    
    
    
    
    #model = RESIDUAL_net_action_1
    
    with tf.device(device):
        
        #change shapes of placeholders
        x = tf.placeholder(tf.float32, [None, 6, 7, 2])
        is_training = tf.placeholder(tf.bool, name='is_training')
        
        
        scores = model(x, is_training)
       
        
        
    
    # Now that our computational graph has been defined we can run the graph
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        feed_dict = {x: x_np, is_training: train_bool}
        scores_np = sess.run(scores, feed_dict=feed_dict)

        return (scores_np)
    


In [ ]:
#label = [1] * 64

def loop_model_score_grad(x_np, y_np, train_bool, model, params1_v):
    
    
    tf.reset_default_graph()
    pipe = test_RESIDUAL_net_action_1([1])
    loop_model_params_set(train_bool, model, params1_v)
    
    
    
    
    
    #model = RESIDUAL_net_action_1
    
    with tf.device(device):
        
        #change shapes of placeholders
        x = tf.placeholder(tf.float32, [None, 6, 7, 2])
        y = tf.placeholder(tf.int32, [None])
        is_training = tf.placeholder(tf.bool, name='is_training')
        
        
        scores = model(x, is_training)
        
        varis = tf.trainable_variables()
        params = varis
        
        grad_params = tf.gradients(scores, params)
        
        
        
    
    # Now that our computational graph has been defined we can run the graph
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        feed_dict = {x: x_np, y: y_np, is_training: train_bool}
        grad_params_np = sess.run(grad_params, feed_dict=feed_dict)

        return grad_params_np
    

    

In [ ]:


def loop_model_params_get(train_bool, model):
    
    
    
    
    
    
    
    
    
    
    
    with tf.device(device):
        
        #change shapes of placeholders
        is_training = tf.placeholder(tf.bool, name='is_training')
        varis = tf.trainable_variables()
        params = varis
        
        
        
    
    # Now that our computational graph has been defined we can run the graph
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        feed_dict = {is_training: train_bool}
        params_np = sess.run(params, feed_dict=feed_dict)

        return params_np
    

    

In [ ]:
def loop_model_params_set(train_bool, model, params_new):
    

    
    
    
    
    
    
    with tf.device(device):
        
        #change shapes of placeholders
        
        
        varis = tf.trainable_variables()
        params = varis
        
        
        
        
    
    # Now that our computational graph has been defined we can run the graph
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        
        
        #params_np = sess.run(params, feed_dict=feed_dict)
        for i in range(len(varis)):
        #for i in range(len(tf.trainable_weights)):

            
            layer = varis[i]  # Select the layer
            #layer = tf.trainable_weights[i]  # Select the layer

            # And modify it explicitly in TensorFlow
            sess.run(tf.assign(layer,  params_new[i]))



#assign_op = x.assign(1)
#sess.run(assign_op)
    

### Training Loop

In [ ]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.
    
    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for
    
    Returns: Nothing, but prints progress during training
    """
    tf.reset_default_graph()    
    with tf.device(device):
        # Construct the computational graph we will use to train the model. We
        # use the model_init_fn to construct the model, declare placeholders for
        # the data and labels
        x = tf.placeholder(tf.float32, [None, 32, 32, 3])
        y = tf.placeholder(tf.int32, [None])
        
        # We need a place holder to explicitly specify if the model is in the training
        # phase or not. This is because a number of layers behaves differently in
        # training and in testing, e.g., dropout and batch normalization.
        # We pass this variable to the computation graph through feed_dict as shown below.
        is_training = tf.placeholder(tf.bool, name='is_training')
        
        # Use the model function to build the forward pass.
        scores = model_init_fn(x, is_training)

        # Compute the loss like we did in Part II
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=scores)
        loss = tf.reduce_mean(loss)
        

        # Use the optimizer_fn to construct an Optimizer, then use the optimizer
        # to set up the training step. Asking TensorFlow to evaluate the
        # train_op returned by optimizer.minimize(loss) will cause us to make a
        # single update step using the current minibatch of data.
        
        # Note that we use tf.control_dependencies to force the model to run
        # the tf.GraphKeys.UPDATE_OPS at each training step. tf.GraphKeys.UPDATE_OPS
        # holds the operators that update the states of the network.
        # For example, the tf.layers.batch_normalization function adds the running mean
        # and variance update operators to tf.GraphKeys.UPDATE_OPS.
        optimizer = optimizer_init_fn()
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            train_op = optimizer.minimize(loss)

    # Now we can run the computational graph many times to train the model.
    # When we call sess.run we ask it to evaluate train_op, which causes the
    # model to update.
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        t = 0
        for epoch in range(num_epochs):
            print('Starting epoch %d' % epoch)
            for x_np, y_np in train_dset:
                feed_dict = {x: x_np, y: y_np, is_training:1}
                loss_np, _ = sess.run([loss, train_op], feed_dict=feed_dict)
                if t % print_every == 0:
                    print('Iteration %d, loss = %.4f' % (t, loss_np))
                    check_accuracy(sess, val_dset, x, scores, is_training=is_training)
                    print()
                t += 1

In [ ]:
"""hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn(inputs, is_training):
    return two_layer_fc_functional(inputs, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.train.GradientDescentOptimizer(learning_rate)

train_part34(model_init_fn, optimizer_init_fn)"""